# **ETL del archivo Homicidios_Hechos**

## **1.Cargando Las Librerías Necesarias**

In [1]:
import pandas as pd #Cargaremos pandas para manejar los datos de nuestros datasets,
import numpy as np #Cargaremos numpy para manejar los datos,
import gzip #Cargaremos gzip para poder abrir archivos con compresion gzip",
import datetime #Cargaremos datetime para manejar fechas",
from google.colab import drive #Cargaremos drive para poder acceder a los archivos de drive"
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
url01="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/homicidios.xlsx"
url02="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/homicidios_hechos.csv"

## **2.Fase ETL**

### 2.1.Carga de Datos

In [3]:
df_homicidios_hechos=pd.read_excel(url01,sheet_name="HECHOS")

In [4]:
df_homicidios_hechos.to_csv(url02,index=True,header=True,encoding='utf-8')

In [5]:
df_homicidios_hechos=pd.read_csv(url02,encoding='utf-8')

In [6]:
df_homicidios_hechos.head()

,Unnamed: 0,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


### 2.2.Analisis Descriptivo De Los Datos

#### 2.2.1.Funciones para el analisis descriptivo de los datos

In [7]:
def check_df(dataset, head = 5):

  """
  Funcion creada para tener una vista general de la base de datos
  """

  print('*'*30 + 'Forma de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.shape, end = '\n'*2)#Tamaño de la base de datos

  print('*'*30 + 'Informacion general de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.info(), end = '\n'*2)# Informacion general

  print('*'*30 + 'NaN cantidad de nulos' + '*'*30, end = '\n'*2)
  print(dataset.isnull().sum(), end = '\n'*2)#Cuenta la cantidad de NaN por columna

#### 2.2.2.Descripcion De Los Datos

In [8]:
check_df(df_homicidios_hechos)

******************************Forma de la base de datos******************************

(696, 22)

******************************Informacion general de la base de datos******************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             696 non-null    int64  
 1   ID                     696 non-null    object 
 2   N_VICTIMAS             696 non-null    int64  
 3   FECHA                  696 non-null    object 
 4   AAAA                   696 non-null    int64  
 5   MM                     696 non-null    int64  
 6   DD                     696 non-null    int64  
 7   HORA                   696 non-null    object 
 8   HH                     696 non-null    object 
 9   LUGAR_DEL_HECHO        696 non-null    object 
 10  TIPO_DE_CALLE          696 non-null    object 
 11  Calle          

### 2.3.Transformacion De Los Datos

#### 2.3.1.Funciones para la transformacion de los datos

In [9]:
def Transform_Hora(text):

  '''

  La funcion de Transform_Hora recibe como parametro un dato de cualquier tipo y
  devuelve un objeto de tipo datetime extrayendo la hora del dato.
  Observando 2 formatos de fecha y un tipo de dato string entonces procedemos de
  la siguiente manera :
  format_full='%Y-%m-%d %H:%M:%S'
  format_short='%H:%M:%S'
  definiremos el formato que se presentan y se extraera posteriormente

  '''
  text=str(text)
  format_full='%Y-%m-%d %H:%M:%S'
  format_short='%H:%M:%S'
  if text.count("-")>=2:
    text.replace("1900","2018")
    datetime_object = datetime.datetime.strptime(text, format_full).time()

  elif text.count(":")>=1:
    datetime_object = datetime.datetime.strptime(text, format_short).time()

  else:
    datetime_object="SD"
  return  datetime_object


def Transform_Dir(text:str):
  """

  Esta funcion esta diseñada para reordenar la columna LUGAR_DEL_HECHO teniendo
  como principla estructura teniendo como prefijo AV. y DR. y eliminando la coma
  normaliando las abreviaciones en general

  """
  text=str(text).upper()
  if "," in text:
    parts = [part.strip() for part in text.split(",")]
    if len(parts) == 2:
        text= f"{parts[1]} {parts[0]} "
    if len(parts) == 3:
        text= f"{parts[2]} {parts[1]} {parts[0]} "
  if "DR." in text:
    parts = [part.strip() for part in text.split("DR.")]

    if len(parts) == 2:
      text= f"DR. {parts[0]} {parts[1]}"

  if "AV." in text:
    parts = [part.strip() for part in text.split("AV.")]

    if len(parts) == 2:
      text= f"AV. {parts[0]} {parts[1]}"



  else:
    text=""

  text=text.replace('GRAL.','GENERAL').replace('TTE.','TENIENTE').replace('CNEL.','CORONEL')
  text=text.strip()

  return text

#### 2.3.2.Transformacion de los datos

Se reviso las primeras las primeras 5  columnas **'ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD'** y no muestran errores como tal

In [10]:
df_homicidios_hechos.drop(columns=['Unnamed: 0'],inplace=True)

In [12]:
df_homicidios_hechos.columns=['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'CALLE', 'ALTURA', 'CRUCE',
       'DIRECCION NORMALIZADA', 'COMUNA', 'XY (CABA)', 'LONGITUD', 'LATITUD',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO']

In [13]:
df_homicidios_hechos['HORA']=df_homicidios_hechos['HORA'].apply(Transform_Hora)

In [14]:
df_homicidios_hechos[df_homicidios_hechos['HORA']=="SD"]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,ALTURA,CRUCE,DIRECCION NORMALIZADA,COMUNA,XY (CABA),LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,SD,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Observamos que existe aun el dato SD en la columna **HORA** por tanto procedemos a cambiarlo por la moda de la columna en la **comuna** No 11

In [15]:
df_homicidios_hechos[['HORA','HH']][df_homicidios_hechos['COMUNA']==11].mode()

,HORA,HH
0,07:30:00,6
1,11:00:00,NaN


In [16]:
df_homicidios_hechos['HORA']=df_homicidios_hechos['HORA'].apply(lambda x: datetime.time(7, 30) if x=='SD' else x)

In [17]:
df_homicidios_hechos[df_homicidios_hechos['HH']=='SD']

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,ALTURA,CRUCE,DIRECCION NORMALIZADA,COMUNA,XY (CABA),LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,07:30:00,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


In [18]:
df_homicidios_hechos['HH']=df_homicidios_hechos['HH'].apply(lambda x: '7' if x=='SD' else x)

In [19]:
df_homicidios_hechos['HH']=pd.to_numeric(df_homicidios_hechos['HH'], errors='coerce', downcast='integer')

In [20]:
df_homicidios_hechos['TIPO_DE_CALLE']=df_homicidios_hechos['TIPO_DE_CALLE'].apply(lambda x: 'AV GENERAL PAZ' if x=='GRAL PAZ' else x)

In [22]:
df_homicidios_hechos['CALLE']=df_homicidios_hechos['CALLE'].apply(Transform_Dir)

In [23]:
df_homicidios_hechos['CRUCE']=df_homicidios_hechos['CRUCE'].apply(Transform_Dir)

In [25]:
df_homicidios_hechos['DIRECCION NORMALIZADA'] = df_homicidios_hechos.apply(
    lambda columna: (columna["CALLE"] + " Y " + columna["CRUCE"]) if (columna["CRUCE"] != "" and columna["CALLE"] != "")
    else (columna["CALLE"] if columna["CALLE"] != ""
          else (columna["CRUCE"] if columna["CRUCE"] != ""
                else columna["LUGAR_DEL_HECHO"]))
    ,axis=1)


In [26]:
df_homicidios_hechos['COMUNA']=pd.to_numeric(df_homicidios_hechos['COMUNA'], errors='coerce', downcast='integer')

In [28]:
df_homicidios_hechos.drop(columns=['LUGAR_DEL_HECHO','ALTURA','XY (CABA)'],inplace=True)

In [29]:
df_homicidios_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,CALLE,CRUCE,DIRECCION NORMALIZADA,COMUNA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AVENIDA,AV. PIEDRA BUENA,AV. GENERAL F. FERNANDEZ DE LA CRUZ,AV. PIEDRA BUENA Y AV. GENERAL F. FERNANDEZ DE...,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GENERAL PAZ,AV. GENERAL PAZ,AV. DE LOS CORRALES,AV. GENERAL PAZ Y AV. DE LOS CORRALES,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AVENIDA,AV. ENTRE RIOS,,AV. ENTRE RIOS,1,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AVENIDA,AV. LARRAZABAL,,AV. LARRAZABAL,8,-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AVENIDA,AV. SAN JUAN,,AV. SAN JUAN,1,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


### 3.Exportacion

In [30]:
path="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Data_Clean/homicidios_hechos.parquet.gzip"

In [31]:
df_homicidios_hechos.to_parquet(path,compression="gzip")